In [3]:
!pip install transformers==2.11.0 --quiet
!pip install simpletransformers==0.41.0 --quiet
#!pip install nlpaug numpy matplotlib python-dotenv --quiet
!pip install pyspellchecker --quiet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import random
import torch
from simpletransformers.classification import ClassificationModel
import pandas as pd
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax
import sklearn
from sklearn.metrics import log_loss, f1_score
import numpy as np
import random
import numpy as np
import torch

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopwords=stopwords.words('english')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('using GPU')
else:
    device = torch.device("cpu")
    print('using CPU')

using GPU


In [7]:
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(79)

In [8]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

Shape of train data :  (7613, 5)
Shape of test data :  (3263, 4)


# 文本清洗

去除url

In [9]:
import re
url=re.compile('https?://\S+|www\.\S+')
for i in range(train.shape[0]):
    train['text'][i]=url.sub(r'',train['text'][i])
for i in range(test.shape[0]):
    test['text'][i]=url.sub(r'',test['text'][i])

去除html

In [10]:
html_tag=re.compile(r'<.*?>')
for i in range(train.shape[0]):
    train['text'][i]=html_tag.sub(r'',train['text'][i])
for i in range(test.shape[0]):
    test['text'][i]=html_tag.sub(r'',test['text'][i])

emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
for i in range(train.shape[0]):
    train['text'][i]=emoji_pattern.sub(r'',train['text'][i])
for i in range(test.shape[0]):
    test['text'][i]=emoji_pattern.sub(r'',test['text'][i])

将有意义的特殊符号与英文表达中的缩写转换成英文单词

In [11]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

将单词都转成小写

In [12]:
from nltk import word_tokenize,PorterStemmer

wnl=PorterStemmer()
train_token=[]
test_token=[]
for text in train['text']:
    tokens=word_tokenize(text)
    tokens=[convert_abbrev(word) for word in tokens]
    train_token.append([t.lower() for t in tokens])
for text in test['text']:
    tokens=word_tokenize(text)
    tokens=[convert_abbrev(word) for word in tokens]
    test_token.append([t.lower() for t in word_tokenize(text)])

去除标点符号和停用词

In [13]:
import string
english_punctuations = set(string.punctuation)
stopwords=set(stopwords)
stopwords_n_punctuations=english_punctuations|stopwords
#处理训练集
for i in range(len(train_token)):
        train_token[i]=[t for t in train_token[i] if t not in stopwords_n_punctuations and t[0]!="'"]
#处理测试集
for i in range(len(test_token)):
        test_token[i]=[t for t in test_token[i] if t not in stopwords_n_punctuations and t[0]!="'"]
      
train_data=[' '.join(token) for token in train_token]
test_data=[' '.join(token) for token in test_token]

In [14]:
# Remove the mislabelled tweets
#incorrect_labels_df = train.groupby(['text']).nunique().sort_values(by='target', ascending=False)
#incorrect_labels_df = incorrect_labels_df[incorrect_labels_df['target'] > 1]
#incorrect_texts = incorrect_labels_df.index.tolist()
#train = train[~train.text.isin(incorrect_texts)]

# 合并地点和文本

In [15]:

# Add the keyword column to the text column
train['location'].fillna('', inplace=True)
train['loctext'] = train['location'] + ' ' + train['text'] 
test['location'].fillna('', inplace=True)
test['loctext'] = test['location'] + ' ' + test['text'] 

In [16]:
train.head(50)

,id,keyword,location,text,target,loctext
0,1,NaN,,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake ...
1,4,NaN,,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,NaN,,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are...
3,6,NaN,,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation o..."
4,7,NaN,,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as...
5,8,NaN,,#RockyFire Update => California Hwy. 20 closed...,1,#RockyFire Update => California Hwy. 20 close...
6,10,NaN,,#flood #disaster Heavy rain causes flash flood...,1,#flood #disaster Heavy rain causes flash floo...
7,13,NaN,,I'm on top of the hill and I can see a fire in...,1,I'm on top of the hill and I can see a fire i...
8,14,NaN,,There's an emergency evacuation happening now ...,1,There's an emergency evacuation happening now...
9,15,NaN,,I'm afraid that the tornado is coming to our a...,1,I'm afraid that the tornado is coming to our ...


In [17]:
train=train.drop(['id'],axis=1)
train=train.drop(['keyword'],axis=1)
train=train.drop(['text'],axis=1)
train=train.drop(['location'],axis=1)
train.head()

,target,loctext
0,1,Our Deeds are the Reason of this #earthquake ...
1,1,Forest fire near La Ronge Sask. Canada
2,1,All residents asked to 'shelter in place' are...
3,1,"13,000 people receive #wildfires evacuation o..."
4,1,Just got sent this photo from Ruby #Alaska as...


In [18]:
first_col = ['loctext']
last_cols = [col for col in train.columns if col not in first_col]

train = train[first_col+last_cols]
train.head()

,loctext,target
0,Our Deeds are the Reason of this #earthquake ...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are...,1
3,"13,000 people receive #wildfires evacuation o...",1
4,Just got sent this photo from Ruby #Alaska as...,1


In [19]:
final=pd.DataFrame()
final['id']=test['id']
final.head()

,id
0,0
1,2
2,3
3,9
4,11


In [20]:
test=test.drop(['id'],axis=1)
test=test.drop(['keyword'],axis=1)
test=test.drop(['text'],axis=1)
test=test.drop(['location'],axis=1)
test['label']=0
test.head()

,loctext,label
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, ...",0
2,"there is a forest fire at spot pond, geese ar...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,0


In [21]:
first_col = ['loctext']
last_cols = [col for col in test.columns if col not in first_col]

test = test[first_col+last_cols]
test.head()

,loctext,label
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, ...",0
2,"there is a forest fire at spot pond, geese ar...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,0


In [22]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [23]:
test.head()

,loctext,label
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, ...",0
2,"there is a forest fire at spot pond, geese ar...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,0


# 查看训练权重

In [24]:
4313/3245

1.3291217257318952

In [25]:
train = train.reindex(np.random.permutation(train.index))
train= train.reset_index(drop=True)
train.head()

,loctext,target
0,#ModiMinistry Madhya Pradesh Train Derailment...,1
1,"Yeezy Taught Me , NV @KimKardashian can you pl...",0
2,wo Pic of 16yr old PKK suicide bomber who det...,1
3,@OKgooner hahaha great song. 'Spent 15 years ...,0
4,"Hawthorne, NE PM Abe pledged to make every eff...",1


In [26]:
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax

In [27]:
f1=sklearn.metrics.f1_score

# 模型参数

In [28]:
# model configuration
model_args = { 
    "save_eval_checkpoints": False,
     "save_model_every_epoch": False,
      'reprocess_input_data': True,
       'overwrite_output_dir': True,
       'manual_seed': 79,
       "silent": True, 
    'num_train_epochs': 2,
    'learning_rate': 2e-5,
    'fp16': False,
    'max_seq_length': 64,


}

#'max_seq_length': 128,
#            'evaluate_during_training': True,
 #   'evaluate_during_training_steps': 1000,
  #    'use_early_stopping': True,
   # 'early_stopping_patience': 3,

# 模型训练

In [29]:
%%time
torch.cuda.empty_cache()
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=79)#10折交叉验证
err=[]
y_pred_tot=[]
for train_index, test_index in kf.split(train, train['target']):
  train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
  model_rb = ClassificationModel('roberta', 'roberta-base', weight=[1, 1.329], args=model_args)#训练权重1：1.329
  model_rb.train_model(train1_trn, eval_df=train1_val)
  result, model_outputs, _ = model_rb.eval_model(train1_val, f1=sklearn.metrics.f1_score, acc=sklearn.metrics.accuracy_score)
  print(f"f1 score:{result['f1']}", f"Accuracy:{result['acc']}")    
  err.append((result['f1']))
  predictions, _ = model_rb.predict(test['loctext'])
  y_pred_tot.append(predictions)
print("Mean f1 score: ",np.mean(err))


f1 score:0.8168168168168168 Accuracy:0.8398950131233596
f1 score:0.7922077922077922 Accuracy:0.8320209973753281
f1 score:0.8006430868167203 Accuracy:0.8372703412073491
f1 score:0.7929936305732485 Accuracy:0.8291721419185283
f1 score:0.7772657450076805 Accuracy:0.80946123521682
f1 score:0.8118195956454121 Accuracy:0.8409986859395532
f1 score:0.7656765676567657 Accuracy:0.8134034165571616
f1 score:0.838006230529595 Accuracy:0.8633377135348226
f1 score:0.8198757763975155 Accuracy:0.8475689881734559
f1 score:0.788961038961039 Accuracy:0.8291721419185283
Mean f1 score:  0.8004266280612585
CPU times: user 27min 56s, sys: 37.9 s, total: 28min 34s
Wall time: 30min 44s


In [34]:
to_submit =np.mean(y_pred_tot,0)

In [35]:
to_submit

array([1., 1., 1., ..., 1., 1., 1.])

In [36]:
#predictions, raw_outputs = model_rb.predict(test['final_text'])
final['target']=to_submit
final['target'] = final['target'].apply(lambda x: 1 if x>0.5 else 0)
final.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [37]:
final.to_csv('submit2.csv',index=False)